# TODO : Intro

## Install SAP Cloud SDK for AI
For this workshop we will use the SAP Cloud SDK for AI : Python. SAP Cloud SDK for AI is the official SDFk for SAP AI core, generative AI Hub, and orchestration. Overall the SDK composed of three python disctribution:
* You can use **sap-ai-sdk-base** to access the AI API using Python methods and data structures.
* You can use **sap-ai-sdk-core** to interact with SAP AI Core for administration and public lifecycle management. We will need this later to create object store secrets. 
* You can use **sap-ai-sdk-gen** to integrate native SDK libraries and langchain for accessing models and leverage the orchestration service of generative AI hub with capabilities such as templating, grounding, data masking, and content filtering. For more information, see Generative AI Hub.



In [ ]:

! pip install sap-ai-sdk-gen
! pip install sap-ai-sdk-core 

## Warm up: Ground your LLM with SAP Help
Let us get started with a quick form up by grounding you LLM with SAP Help. 

In [2]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.config import GroundingModule, OrchestrationConfig
from gen_ai_hub.orchestration.models.document_grounding import DocumentGrounding, DocumentGroundingFilter
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.service import OrchestrationService

In [3]:
llm = LLM(
    name="gemini-1.5-flash",
    parameters={
        'temperature': 0.0,
    }
)
template = Template(
            messages=[
                SystemMessage("You are a helpful translation assistant."),
                UserMessage("""Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                Context:{{ ?grounding_response }}
                """),
            ]
        )

In [4]:
# Set up Document Grounding
filters = [
            DocumentGroundingFilter(id="SAPHelp", data_repository_type="help.sap.com")
        ]

grounding_config = GroundingModule(
            type="document_grounding_service",
            config=DocumentGrounding(input_params=["user_query"], output_param="grounding_response", filters=filters)
        )

config = OrchestrationConfig(
    template=template,
    llm=llm,
    grounding=grounding_config
)

In [ ]:
import importlib

orchestration_service = OrchestrationService(
    api_url="https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d002ccdb23f8195c",
    config=config
)

response = orchestration_service.run(
    template_values=[
        TemplateValue( 
            name="user_query",
            value="What roles do I need to access the SAP AI Launchpad ?"
        )
    ]
)

print(response.orchestration_result.choices[0].message.content)

## Ground your LLM with Custom Documents

Besides grounding your LLM with SAP help you can of course ground your LLM with you own documents. The document file that the grounding service support here are PDF, HTML, TXT, JPEG, JPG, DOCX, PNG, TIFF, PPT.
However to use Grounding you need to prepare a knowlegde base in advance. Generative AI hub provides two otions for the users to provide data (prepare knowledege base):

* Option 1: Provide the chunks of document via Vector API directly
* Option 2: Upload the documents to supported data repository and run the data pipeline to vectorize the documents

In the following you will get to know both option, and we start with option 1

### Provide the chunks of documents via Vector API directly

In the following we will do the following steps:
+ Prepare the Vector Knowledge Base
    1. Create Collection
    2. Create documents by directly using the chunks of data provided by users
    3.  Store data in the vector database
    4.  Assign repository IDs to access the data
* Configure Grounding Module in the Orchestration 
    1. Create a grounding request configuration in the orchestration pipeline using repository IDs
    2.  Run the orchestration pipeline and check that the response refers to the user data

In [ ]:
# need Vector API example --

### Upload the documents to the supported data repository and run data pipeline

#### Create generic Secrets for Object Store 
Generic secrets securely store AWS S3 credentials required fro document access

In [9]:
#Get the Service Key and Instantiate AI API Client
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
import os, json
config_file = "/Users/d054176/Downloads/aicore_service_key.json"
with open(config_file,'rb') as config:
    config_json = json.load(config)
    config.close()

base_url = config_json['serviceurls']['AI_API_URL']
client_id = config_json['clientid']
client_secret = config_json['clientsecret']
auth_url = config_json['url']



client = AICoreV2Client (base_url=base_url+'/v2',
                         auth_url= auth_url+'/oauth/token',
                         client_id=client_id,
                         client_secret=client_secret,
                         resource_group="default"
                         )


In [ ]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
import os, json
config_file = "/Users/d054176/Downloads/objectstore_skey.json"

with open(config_file,'rb') as config:
    config_json = json.load(config)
    config.close()

access_key = config_json['access_key_id']
secret = config_json['secret_access_key']
bucket = config_json['bucket']
object_store_secret_create = client.object_store_secrets.create( 
    name = "aws3-secret",
    type = "S3",
    bucket= bucket,
    endpoint="s3-eu-central-1.amazonaws.com",
    path_prefix= "ai167_grounding" ,
    region="eu-central-1",
    data={"AWS_ACCESS_KEY_ID":access_key,"AWS_SECRET_ACCESS_KEY":secret}
)
secret_get = client.object_store_secrets.get(name="aws3-secret").metadata
print(secret_get)

#### Create Data Pipeline

In [ ]:
from gen_ai_hub.proxy import get_proxy_client
from gen_ai_hub.document_grounding.client import PipelineAPIClient
from gen_ai_hub.document_grounding.models.pipeline import S3PipelineCreateRequest, CommonConfiguration

aicore_client = get_proxy_client();
print(aicore_client.resource_group) 
print(aicore_client.base_url)

pipeline_api_client = PipelineAPIClient(aicore_client)

generic_secret_s3_bucket = "aws3-secret"
s3_config = S3PipelineCreateRequest(configuration= CommonConfiguration(destination=generic_secret_s3_bucket))
response = pipeline_api_client.create_pipeline(s3_config)

print(f"Reference the Vector knowledge base using the pipeline ID: {response.pipelineId}")
# check the status of the vectorization pipeline until it is completed
print(pipeline_api_client.get_pipeline_status(response.pipelineId))